In [5]:
import cplex

# 将城市节点数量与他们之间的距离表示出来
number = 4
nodes = 3
my_obj = [0, 20, 50, 70, 20, 0, 30, 80, 50, 30, 0, 100, 70, 80, 100, 0, 0, 0, 0, 0]
# 变量的名称
b_var = ["x11", "x12", "x13", "x14", "x21", "x22", "x23", "x24", "x31", "x32", "x33", "x34",
         "x41", "x42", "x43", "x44"]
b_var_ind_format = [["x11", "x12", "x13", "x14"], ["x21", "x22", "x23", "x24"], ["x31", "x32", "x33", "x34"],
                    ["x41", "x42", "x43", "x44"]]
c_var = ["u1", "u2", "u3", "u4"]
my_sense_b = "E" * 12  # 限流变量约束条件类型定义
my_sense_c = "L" * 6  # 连续变量约束条件类型定义
my_rownames_b = ["r1", "r2", "r3", "r4", "r5", "r6", "r7", "r8", "r9", "r10", "r11", "r12"]  # 约束条件的名称
my_rownames_c = []
list_c = []  # 存储约束条件中变量和他们系数的小列表
rows_c = []
result = []
# 下面是一个构建连续变量约束条件系数矩阵的循环程序
k = 0
for i in range(1, 4):
    for j in range(1, 4):
        if i != j:
            list_c.append([c_var[i], c_var[j], b_var_ind_format[i][j]])
            # 上面是完成了一个变量的小组合
            list_c.append([1, -1, number])
            rows_c.append([list_c[k], list_c[k + 1]])
            k += 2
rhs_n = len(rows_c)
print(rhs_n)
print("定义连续变量的约束条件为:", rows_c)
# rhs = [float(nodes)] * int(rhs_n)
# print(len(rhs)),这些都是找错误用的
# 下面构建连续约束条件的名字
for i in range(int(rhs_n)):
    my_rownames_c.append("c" + str(i + 1))


def tsp_method(prob):
    """
    :param prob:代表建立起的cplex实例
    :return: 无返回值
    实现变量的添加，设置好约束条件了。针对旅行商问题我们设置的变量为0-1变量，约束条件为MZT模型
    """
    prob.objective.set_sense(prob.objective.sense.minimize)  # 目标函数的意义

    prob.variables.add(names=b_var + c_var, types="B" * 16 + "C" * 4, obj=my_obj)  # 定义变量名，定义0-1类型，定义目标函数系数
    # prob.variables.add(names=c_var, types="C" * 4)  # 定义连续变量，消除子环

    rows = [[["x21", "x31", "x41"], [1.0, 1.0, 1.0]],
            [["x12", "x32", "x42"], [1.0, 1.0, 1.0]],
            [["x13", "x23", "x43"], [1.0, 1.0, 1.0]],
            [["x14", "x24", "x34"], [1.0, 1.0, 1.0]],
            [["x12", "x13", "x14"], [1.0, 1.0, 1.0]],
            [["x21", "x23", "x24"], [1.0, 1.0, 1.0]],
            [["x31", "x32", "x34"], [1.0, 1.0, 1.0]],
            [["x41", "x42", "x43"], [1.0, 1.0, 1.0]],
            [["x11"], [1.0]],
            [["x22"], [1.0]],
            [["x33"], [1.0]],
            [["x44"], [1.0]]]  # 这个是定义限制流量的约束条件的系数
    rhs = [float(nodes)] * int(rhs_n)
    prob.linear_constraints.add(lin_expr=rows + rows_c, senses=my_sense_b + my_sense_c,
                                rhs=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0] + rhs,
                                names=my_rownames_b + my_rownames_c)  # 建立好约束条件
    # prob.linear_constraints.add(lin_expr=rows_c, senses=my_sense_c,
    #                             rhs=[float(nodes)] * int(rhs_n),
    #                             names=my_rownames_c)


my_prob = cplex.Cplex()
handle = tsp_method(my_prob)
my_prob.solve()

# except CplexError as exc:
#     print("异常就是这个", exc)
print()
# solution.get_status() returns an integer code
print("Solution status = ", my_prob.solution.get_status(), ":", end='')

# the following line prints the corresponding string
print(my_prob.solution.status[my_prob.solution.get_status()])
# print("Solution value  = ", my_prob.solution.get_objective_value())  # 这个函数是返回目标函数值

numcols = my_prob.variables.get_num()
numrows = my_prob.linear_constraints.get_num()

x = my_prob.solution.get_values()

print("x: ")
print(x)
print('1', end='')
for i in range(len(x)):
    if x[i] == 1 and i < len(x) - 4:  # 因为后面四个变量都是ui所以不把他们作为路径点统计
        print("->", b_var[i][2], end='')


6
定义连续变量的约束条件为: [[['u2', 'u3', 'x23'], [1, -1, 4]], [['u2', 'u4', 'x24'], [1, -1, 4]], [['u3', 'u2', 'x32'], [1, -1, 4]], [['u3', 'u4', 'x34'], [1, -1, 4]], [['u4', 'u2', 'x42'], [1, -1, 4]], [['u4', 'u3', 'x43'], [1, -1, 4]]]
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 5 columns.
Reduced MIP has 14 rows, 15 columns, and 42 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Found incumbent of value 220.000000 after 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 14 rows, 15 columns, and 42 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 11.
MIP emphasis: balance optimality and feasibility.
MIP search 

In [6]:
# -*- coding: utf-8 -*-
# The MIP problem solved in this example is:
#  (2,6),(2,3),(6,5),(5,4),(4,6)，：：(2,1),(3,3),(6,9),(5,7),(4,5)，：：(2,2),(2,4.5),(6,7.5),(5.5,4。5),(7.4,8.6）
#   Maximize  6x1 + 3 x2 + 5 x3 + 4 x4 +6 x5+ x6 +3x7 + 9 x8+7 x9+5 x10+ 2 x11 +4.5 x12+7.5 x13 + 4.5x14 +8.6x15= 价值最大化
#   Subject to
#      (2,6),(2,3),(6,5),(5,4),(4,6)，：：(2,1),(3,3),(6,9),(5,7),(4,5)，：：(2,2),(2,4.5),(6,7.5),(5.5,4。5),(7.4,8.6）      重量小于背包的盛重量
#       2x1 + 2 x2 + 6 x3 + 5 x4 +4 x5+ 2 x6 +3x7 + 6 x8+7 x9+5 x10+ 2 x11 +2 x12+6 x13 + 5.5 x14 +7.4x15
#      -x2+ x5<=0                2号背包必须在五号背包之前放入
#   Bounds             对于各个背包要么取要么不取0 1二值问题

#        0 <= x1 <= 1
#        0 <= x2 <= 1
#        0 <= x3 <= 1
#        0 <= x4 <= 1
#        0 <= x5 <= 1
#        0 <= x6 <= 1
#        0 <= x7 <= 1
#        0 <= x8 <= 1
#        0 <= x9 <= 1
#        0 <= x10 <= 1
#        0 <= x11 <= 1
#        0 <= x12 <= 1
#        0 <= x13 <= 1
#        0 <= x14 <= 1
#        0 <= x15 <= 1
#   Integers
#       x1,x2,x3,x4,x5，x6,x7,x8,x9,x10,x11,x12,x13,x14,x15

import cplex
from cplex.exceptions import CplexError
# (2,6),(2,3),(6,5),(5,4),(4,6)，：：(2,1),(3,3),(6,9),(5,7),(4,5)，：：(2,2),(2,4.5),(6,7.5),(5.5,4.5),(7.4,8.6）
# data common to all populateby functions
my_obj = [6.0, 3.0, 5.0, 4.0, 6.0, 1.0, 3.0, 9.0, 7.0, 5.0, 2.0, 4.5, 7.5, 4.5, 8.6]
my_ub = [1.0, 1.0, 1.0, 1.0, 1.0,1.0, 1.0, 1.0, 1.0, 1.0,1.0, 1.0, 1.0, 1.0, 1.0]
my_lb = [0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0]
my_ctype = "IIIIIIIIIIIIIII"     #表示参数的类型 c应该表示数值，I表示整数
my_colnames = ["x1", "x2", "x3", "x4", "x5","x6", "x7", "x8", "x9", "x10","x11", "x12", "x13", "x14", "x15"]  #表示实验的



my_rhs = [40,0,0]
my_rownames = ["r1", "r2","r3"]
my_sense = 'LLL'


def populatebyrow(prob):
  prob.objective.set_sense(prob.objective.sense.maximize)
  # 指要优化的目标函数是要求最大化
  prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                     names=my_colnames)

  rows = [[["x1", "x2", "x3", "x4", "x5","x6", "x7", "x8", "x9", "x10","x11", "x12", "x13", "x14", "x15"], [2.0, 2.0, 6.0, 5.0, 4.0,2.0, 3.0, 6.0, 5.0, 4.0,2.0,2.0,6.0,5.5,7.4 ]],
          [["x2","x5"],[-1,1]],
          [["x7","x12"],[-1, 1]]
          ]
  # 上面是背包问题所要求的，后面是所要附加的约束条件，即要求2号背包必须在五号背包之前放入
  prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                              rhs=my_rhs, names=my_rownames)


try:
  my_prob = cplex.Cplex()
  handle = populatebyrow(my_prob)
  my_prob.solve()

except CplexError as exc:
  print(exc)

print()
# solution.get_status() returns an integer code
print("Solution status = ", my_prob.solution.get_status(), ":", end=' ')
# the following line prints the corresponding string
print(my_prob.solution.status[my_prob.solution.get_status()])
print("Solution value  = ", my_prob.solution.get_objective_value())

numcols = my_prob.variables.get_num()
numrows = my_prob.linear_constraints.get_num()

slack = my_prob.solution.get_linear_slacks()
x = my_prob.solution.get_values()

print('x: ')
print(x)


Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 2 rows and 2 columns.
MIP Presolve modified 16 coefficients.
Reduced MIP has 5 rows, 17 columns, and 26 nonzeros.
Reduced MIP has 15 binaries, 2 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2 rows and 2 columns.
MIP Presolve added 3 rows and 3 columns.
Reduced MIP has 6 rows, 18 columns, and 31 nonzeros.
Reduced MIP has 15 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 2.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0

In [10]:
# 2.计算  (2+x1)/(1+x2) - 3*x1+4*x3 的最小值  x1,x2,x3的范围都在0.1到0.9之间

from scipy.optimize import minimize
import numpy as np

def fun(args):
    a,b,c,d=args
    v=lambda x: max(x)
    return v

def con(args):
    # 约束条件 分为eq 和ineq
    #eq表示 函数结果等于0 ； ineq 表示 表达式大于等于0  
    x1min, x1max, x2min, x2max,x3min,x3max = args
    cons = ({'type': 'ineq', 'fun': lambda x: x[0] - x1min},\
              {'type': 'ineq', 'fun': lambda x: -x[0] + x1max},\
             {'type': 'ineq', 'fun': lambda x: x[1] - x2min},\
                {'type': 'ineq', 'fun': lambda x: -x[1] + x2max},\
            {'type': 'ineq', 'fun': lambda x: x[2] - x3min},\
             {'type': 'ineq', 'fun': lambda x: -x[2] + x3max})
    return cons

if __name__ == "__main__":
    #定义常量值
    args = (2,1,3,4)  #a,b,c,d
    #设置参数范围/约束条件
    args1 = (0.2,0.9,0.1, 0.9,0.1,0.9)  #x1min, x1max, x2min, x2max
    cons = con(args1)
    #设置初始猜测值  
    x0 = np.asarray((0.5,0.5,0.5))    
    res = minimize(fun(args), x0, method='SLSQP',constraints=cons)
    print(res.fun)
    print(res.success)
    print(res.x)

0.1999999999999995
True
[0.2 0.1 0.1]
